In [ ]:
# %load_ext pretty_jupyter

In [ ]:
import sys
from pathlib import Path
import scvi
import cellxgene_census
import numpy as np
import pandas as pd
from scipy.stats import mode
import scanpy as sc
import sklearn
import warnings
import faiss
sys.path.insert(0, "../")
import scgpt as scg


# scGPT

In [29]:
model_dir = Path("../resources/scGPT_human")
adata = sc.read_h5ad("../save/adata.h5ad")
cell_type_key = "scGPT_metainfo"
gene_col = "index"
adata.obs[cell_type_key] = 0

adata_sub = sc.pp.subsample(data=adata, fraction=0.01, copy=True)

In [ ]:
adata_sub_embed = scg.tasks.embed_data(
    adata_sub,
    model_dir,
    gene_col=gene_col,
    obs_to_save=cell_type_key,  # optional arg, only for saving metainfo
    batch_size=98,
    return_new_adata=True,
)

adata_sub_emebd = adata_sub_embed.X


In [ ]:
%%time

from build_atlas_index_faiss import load_index, vote

use_gpu = faiss.get_num_gpus() > 0
index, meta_labels = load_index(
    index_dir="../resources/CellXGene_faiss_index/",
    use_config_file=False,
    use_gpu=False,
)
print(f"Loaded index with {index.ntotal} cells")


k = 50
# test with the first 100 cells
distances, idx = index.search(adata_sub_emebd, k)

predict_labels = meta_labels[idx]
from scipy.stats import mode
from tqdm import tqdm

voting = []
for preds in tqdm(predict_labels):
    voting.append(vote(preds, return_prob=False)[0])
voting = np.array(voting)


adata_sub.obs["scGPT"] = voting

In [ ]:
adata_sub.obs["manual_annotation"].value_counts()

In [ ]:
adata_sub.obs["scGPT"].value_counts()

In [ ]:
from matplotlib import rc_context
import matplotlib.pyplot as plt
with rc_context({"figure.figsize": (10,10)}):
    sc.pl.umap(adata_sub, color=["scGPT", "manual_annotation"])
    plt.tight_layout()


# scVI + cellxgene

In [16]:
import cellxgene_census
import cellxgene_census.experimental


In [17]:
census_version = "2023-12-15"
organism = "homo_sapiens"

census = cellxgene_census.open_soma(census_version=census_version)

scvi_info = cellxgene_census.experimental.get_embedding_metadata_by_name(
    embedding_name="scvi",
    organism=organism,
    census_version=census_version,
)


In [20]:
import warnings

warnings.filterwarnings("ignore")

import anndata
import cellxgene_census
import numpy as np
import scanpy as sc
import scvi
from sklearn.ensemble import RandomForestClassifier


In [56]:
!mkdir -p data
!wget -nv -O data/pbmc3k_filtered_gene_bc_matrices.tar.gz http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
!tar -xzf data/pbmc3k_filtered_gene_bc_matrices.tar.gz -C data/


2024-06-20 15:17:22 URL:https://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz [7621991/7621991] -> "data/pbmc3k_filtered_gene_bc_matrices.tar.gz" [1]


In [57]:
adata = sc.read_10x_mtx("data/filtered_gene_bc_matrices/hg19/", var_names="gene_ids")


In [62]:
census = cellxgene_census.open_soma(census_version="2023-12-15")

# Some PBMC data from these collections
# 1. https://cellxgene.cziscience.com/collections/c697eaaf-a3be-4251-b036-5f9052179e70
# 2. https://cellxgene.cziscience.com/collections/f2a488bf-782f-4c20-a8e5-cb34d48c1f7e
dataset_ids = [
    "fa8605cf-f27e-44af-ac2a-476bee4410d3",
    "3c75a463-6a87-4132-83a8-c3002624394d",
]

adata_census = cellxgene_census.get_anndata(
    census=census,
    measurement_name="RNA",
    organism="Homo sapiens",
    obs_value_filter=f"dataset_id in {dataset_ids}",
    obs_embeddings=["scvi"],
)
adata_census.var.set_index("feature_id", inplace=True)


In [67]:
adata_census.obsm["scvi"]

array([[-2.5390625e-02, -2.1839142e-04, -4.4140625e-01, ...,
        -1.1718750e+00, -2.9563904e-04,  1.1138916e-03],
       [-6.7187500e-01,  8.6975098e-04, -2.4902344e-01, ...,
        -1.8750000e+00,  1.4266968e-03,  2.5024414e-03],
       [ 1.9628906e-01, -2.5787354e-03, -1.7968750e-01, ...,
        -1.0546875e+00, -2.0904541e-03, -4.6691895e-03],
       ...,
       [ 1.0234375e+00, -2.5482178e-03, -7.3046875e-01, ...,
        -4.9218750e-01, -6.6528320e-03,  6.0319901e-05],
       [-5.7031250e-01,  2.7465820e-03,  2.0629883e-02, ...,
        -4.0429688e-01,  4.9743652e-03, -3.6926270e-03],
       [-3.2031250e-01, -6.0119629e-03, -1.1796875e+00, ...,
        -9.0234375e-01,  1.7929077e-03, -4.0283203e-03]], dtype=float32)

In [63]:
adata.obs["dataset_id"] = "QUERY"
# Subset the reference dataset to have a similar number of cells to the query dataset
index_subset = np.random.choice(adata_census.n_obs, size=3000, replace=False)
adata_census_subset = adata_census[index_subset, :]


ValueError: Did not find scvi in `.obsm.keys()`. You need to compute it first.

In [ ]:

adata_combined = anndata.concat([adata_census_subset, adata])
sc.pp.neighbors(adata_combined, n_neighbors=15, use_rep="scvi", metric="correlation")
sc.tl.umap(adata_combined)
sc.pl.umap(adata_combined, color=["dataset_id"])


In [ ]:
rfc = RandomForestClassifier()
rfc.fit(adata_census.obsm["scvi"], adata_census.obs["cell_type"].values)
adata.obs["predicted_cell_type"] = rfc.predict(adata.obsm["scvi"])

# let's get confidence scores
probabilities = rfc.predict_proba(adata.obsm["scvi"])
confidence = np.zeros(adata.n_obs)
for i in range(adata.n_obs):
    confidence[i] = probabilities[i][rfc.classes_ == adata.obs["predicted_cell_type"][i]]


INFO     File ../resources/scvi.model_homo_sapiens/model.pt already downloaded                                     


KeyError: "Values ['ENSG00000233576', 'ENSG00000234667', 'ENSG00000214141', 'ENSG00000248574', 'ENSG00000260089', 'ENSG00000248144', 'ENSG00000273540', 'ENSG00000241532', 'ENSG00000228816', 'ENSG00000255838', 'ENSG00000223621', 'ENSG00000214992', 'ENSG00000250794', 'ENSG00000219249', 'ENSG00000272031', 'ENSG00000230006', 'ENSG00000216740', 'ENSG00000243478', 'ENSG00000270560', 'ENSG00000255700', 'ENSG00000248122', 'ENSG00000254351', 'ENSG00000230418', 'ENSG00000236818', 'ENSG00000255014', 'ENSG00000267333', 'ENSG00000220517', 'ENSG00000217791', 'ENSG00000214342', 'ENSG00000214003', 'ENSG00000227590', 'ENSG00000224948', 'ENSG00000261102', 'ENSG00000242561', 'ENSG00000253502', 'ENSG00000230948', 'ENSG00000187172', 'ENSG00000278910', 'ENSG00000282851', 'ENSG00000281406', 'ENSG00000236323', 'ENSG00000231099', 'ENSG00000272779', 'ENSG00000270270', 'ENSG00000268174', 'ENSG00000268438', 'ENSG00000265631', 'ENSG00000269662', 'ENSG00000269431', 'ENSG00000266575', 'ENSG00000125997', 'ENSG0000027

In [ ]:

vae_q.is_trained = True
latent = vae_q.get_latent_representation()
adata.obsm["scvi"] = latent

# filter out missing features
adata = adata[:, adata.var["gene_names"].notnull().values].copy()
adata.var.set_index("gene_names", inplace=True)

In [ ]:
# -.-|m { input: false, output: false, input_fold: show}

import tomlkit
import scanpy as sc
from anndata import AnnData
import pandas as pd
import numpy as np
import scvi
import seaborn as sns
from pandas import DataFrame

from os import path
import session_info
import logging
from tempfile import TemporaryDirectory 
from os import system

logging.basicConfig(level=logging.ERROR)

scvi.settings.seed = 0
sc.set_figure_params(figsize=(6, 6), frameon=False)


In [ ]:
CELL_TYPIST_MODELS: list[str] = []

In [ ]:
def scvi_cellassign_annotate(adata: AnnData, marker_matrix: DataFrame, subsample: int = 2000):

    import torch
    from scvi.external import CellAssign

    torch.set_float32_matmul_precision("high")

    markers_common = list(set(adata.var_names).intersection(set(marker_matrix.index)))
    adata_annotate = sc.pp.subsample(adata, n_obs = subsample, copy = True)
    
    adata_annotate = adata_annotate[:, list(markers_common)].copy()

    adata_annotate.X = adata_annotate.layers[COUNTS_LAYER]
    lib_size = adata_annotate.layers[COUNTS_LAYER].sum(1)
    adata_annotate.obs["size_factor"] = lib_size / np.mean(lib_size)

    scvi.external.CellAssign.setup_anndata(adata_annotate, size_factor_key="size_factor")
    cellassign_model = CellAssign(adata_annotate, marker_matrix)

    return cellassign_model, adata_annotate


def convert_scHCA(tissue: str, quantile: float = 0.98):
    df = pd.read_csv("../resources/scHCL_DB.csv", index_col=0)
    tissue_df = df.filter(regex=tissue)
    quantiles = tissue_df.quantile(q=quantile, axis=0)
    binary_markers = tissue_df.gt(quantiles, axis=1).astype("int")
    
    # Remove markers where it is all 0 or 1
    binary_markers = binary_markers.loc[~(binary_markers == 0).all(axis=1)]
    binary_markers = binary_markers.loc[~(binary_markers == 1).all(axis=1)]

    return binary_markers


def cell_typist_annotate(adata: AnnData, models: list[str], inplace = True):
    import celltypist
    from celltypist import models as ctypist_models

    ctypist_models.download_models(force_update=True, model=models)


    adata_celltypist = adata.copy()  
    adata_celltypist.X = adata.layers[COUNTS_LAYER]  
    sc.pp.normalize_per_cell(adata_celltypist, counts_per_cell_after=10**4)  
    sc.pp.log1p(adata_celltypist)
    adata_celltypist.X = adata_celltypist.X.toarray()

    for model in models:
        loaded_model = ctypist_models.Model.load(model = model)
        predictions = celltypist.annotate(adata_celltypist, model=loaded_model, majority_voting=True)
        predictions_adata = predictions.to_adata()
        adata.obs["celltypist_"+model+"_label"] = predictions_adata.obs.loc[adata.obs.index, "majority_voting"]
        adata.obs["celltypist_"+model+"_conf_score"] = predictions_adata.obs.loc[adata.obs.index, "conf_score"]

    if not inplace:
        return adata


def annotate_scGPT(adata: AnnData):
    pass



In [ ]:
## Pipeline parameters
with open("../config.toml", "r") as f:
    config = tomlkit.parse(f.read())

In [ ]:
ROOT_DIR = config["basic"]["ANALYSIS_DIR"]
DIR_SAVE = path.join(ROOT_DIR, config["basic"]["DIR_SAVE"])
TISSUE = config["basic"]["TISSUE"]
ANNOTATION_METHOD = config["annotation"]["ANNOTATION_METHOD"]
COUNTS_LAYER = config["normalization"]["COUNTS_LAYER"]
NORMAMALIZATION_LAYER = config["normalization"]["NORMALIZATION_METHOD"]

In [ ]:
#adata = sc.read_h5ad(path.join(DIR_SAVE, "adata.h5ad"))
adata = sc.read_h5ad("../save/marcelo_ref.h5ad")

In [ ]:
if ANNOTATION_METHOD == "scvi_cellassign":
    markers = convert_scHCA(TISSUE)
    model, adata_annotated = scvi_cellassign_annotate(adata, markers, subsample=100)
    model.train(batch_size = 8192, max_epochs=400)

    model.history["elbo_validation"].plot()

    predictions = model.predict()
    predictions.head()

    sns.clustermap(predictions, cmap="viridis")
    adata_annotated.obs["cellassign_predictions"] = predictions.idxmax(axis=1).values


if ANNOTATION_METHOD == "celltypist":
    cell_typist_annotate(adata, CELL_TYPIST_MODELS)